Finding signatures of Volcanic Activity using global temperature changes
=========

_This notebook was developed following the [HIDA Datathlon Virtual Challenge](https://www.eventbrite.de/e/hida-datathon-virtual-challenge-registration-100891958564), held on April 2-3, 2020. The following was developed mostly in the days following that challenge._

Given a set of global temperature data over 1000 years, and volcanic activity data, can we develop tools to accurately identify periods of high volcanic activity using only the temperature information?

With this data, I am taking the opportunity to explore **time series data** a bit. As a particle physicist, we rarely work with time series data, so this is a good opportunity to learn some things about time series data manipulation.

The first thing we need to do is to set up the data, below:

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from scipy import signal

# Helper functions used in other parts of the code
import HelperModules
import EngineeredFeatures

ds = xr.open_dataset('MyChallengePaleo/T2m_R1_ym_1stMill.nc')
T2m_R1 = ds.to_dataframe()

TSI = HelperModules.getForcingData('MyChallengePaleo/Solar_forcing_1st_mill.nc','TSI')
AOD = HelperModules.getForcingData('MyChallengePaleo/Volc_Forc_AOD_1st_mill.nc','AOD')

The Data
===============

The temperature is influenced by *forcings*, so-called because they force the global temperature out of equilibrium, consisting of **solar** and **volcanic** forcing. Volcanic activity is indicated by the aerosol optical depth:
>High values indicate a high sulfate load in the stratosphere, leading to a reflection and absorption of incoming short wave radiation in the stratosphere with the net-effectof a decrease in global mean temperatures.

Solar forcing is measured in units of solar irradiance (Wm$^{-2}$). It has an 11-year cycle, plus more irregular, non-periodic variations. In the plot below, the long-term variation is highlighted by the red line.

In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 5))

# Solar forcing
color = 'gray'
p1 = ax1.plot(TSI['time_yr'],TSI['TSI'],label='solar irradiance',color=color)
ax1.set_xlabel('time (year)')
ax1.set_ylabel('Solar irradiance [Wm$^{-2}$]',color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_xlim(-10,1010)
ax1.set_ylim(1364,1366.5)

# Rolling-average solar forcing
p2 = ax1.plot(TSI['time_yr'],TSI['TSI'].rolling(window=11,center=True).mean(),label='solar irradiance (11-yr mean)',color='tab:red')

# Volcanic forcing
color = 'tab:blue'
ax2 = ax1.twinx()
p3 = ax2.plot(AOD['time_yr'],AOD['AOD'],label='volcanic AOD',color=color)
ax2.set_ylabel('Aerosol Optical Depth',color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim(-0.01,0.7);

labs = [l.get_label() for l in p1+p2+p3]
ax1.legend(p1+p2+p3, labs, loc=1);

The Temperature Data
-----------
Now let's look at the global temperature data. This dataset is constructed by running a 1000-year simulation of global temperatures, giving the solar and volcanic forcings above as an input. It consists of a temperature data point for every longitude and latitude, for each year of the entire 1000-year duration.

Let's plot the **global average temperature** alongside the volcanic and the (long-term) solar activity for comparison:

In [ ]:
fig = plt.figure(figsize=(20, 5))
ax = fig.add_subplot(1, 1, 1)
spacial_averaged = T2m_R1.groupby('time').mean()
p1 = ax.plot(AOD['time_yr'],AOD['AOD'],label='volcanic AOD',color='gray')
p2 = ax.plot(TSI['time_yr'],0.3*(-1363.0+TSI['TSI'].rolling(window=11,center=True).mean()),label='solar irradiance (11-yr mean), arb. units',color='red')
ax.set_ylabel('Aerosol Optical Depth',color='gray')
ax.set_ylim(0,0.9)

ax2 = ax.twinx()
p3 = ax2.plot(np.round(spacial_averaged.index/10000),list(spacial_averaged.values),label='Average global temperature')
ax2.set_ylabel('Average global temperature',color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')
ax2.set_ylim(274.5,278.3)

labs = [l.get_label() for l in p1+p2+p3]
ax.legend(p1+p2+p3, labs, loc=3)

plt.xlabel('time [years]')
plt.show()

Plotting Autocorrelation and Power Spectra of the Temperature Data
==========
A time series is **_stationary_** if the current value is not based in any way on the previous value (e.g. a random walk). From "Forecasting: Principles and Practice":
> In general, a stationary time series will have no predictable patterns in the long-term. Time plots will show the series to be roughly horizontal (although some cyclic behaviour is possible), with constant variance.

Some of the typical tools for determining the nature of time series data include *autocorrelation*, the correlation of the data with a time-delay of itself. If a time series is not random but has an underlying structure, for instance a periodic signal, then an autocorrelation plot might shed light on this behavior. Here is what that looks like for this dataset, below.

From these plots, it is clear that (a) the temperature is clearly correlated with the years immediately preceding, but (b) otherwise there is no discernable pattern in the data -- not even the 11-year solar cycle is visible.

In [ ]:
temporal_averaged = EngineeredFeatures.getGlobalTemperatureAverageSeries(T2m_R1['T2m'])
ta_diff = temporal_averaged.diff().fillna(0)
fig = plt.figure(figsize=(15, 4))
ax1 = fig.add_subplot(1, 2, 1,label='ax1')
pd.plotting.autocorrelation_plot(temporal_averaged,ax=ax1,label='autocorrelation of data')
ax2 = fig.add_subplot(1, 2, 2)
pd.plotting.autocorrelation_plot(ta_diff,ax=ax2,label='autocorrelation of differenced data');

Periodogram (Power Spectrum)
-----------

Similarly, a periodogram (or power spectrum) of the data shows no discernable maxima.

In [ ]:
fig = plt.figure(figsize=(10, 4))
frequencies, power_spectrum = signal.periodogram(temporal_averaged,30)
ax = plt.semilogy(frequencies, power_spectrum)
plt.ylabel('Spectrum')
plt.xlabel('Frequency [year]');
#plt.xlim(8,15)

Differencing the Temperature Data as a way to find volcanic activity
=========
Another way to visualize time series is by *differencing*, e.g. plotting the change in values for each series step. If a series is not _stationary_, then it is possible that the differenced version of the time series is stationary (e.g. a random walk). Differencing is investigated for this dataset below.

Since we know that volcanic activity causes a drop in the temperature, we should look for that. But we don't want to just set a temperature thresold; then low solar activity, or simply a statistical fluctuation, could trigger a false volcanic "alarm". So we use the year-over-year temperature **_difference_** to find large, sudden changes in the temperature. This will work both in relatively cold periods as well as warm ones.

In the end it took a bit of fiddling to find something that looked good (e.g. not too noisy), namely: a **5-year rolling average of temperatures, differenced, that difference itself averaged over 4 years.**

The differencing, shown below, reveals large negative spikes that correlate closely with periods of large volcanic activity. Setting a simple threshold on this quantity is a good predictor of volcanic activity:

In [ ]:
temporal_averaged = EngineeredFeatures.getGlobalTemperatureAverageSeries(T2m_R1['T2m'])
ta_diff = temporal_averaged.rolling(window=5).mean().diff().fillna(0).rolling(window=4).mean()

def indexToYear(index) :
    return np.round(index/10000)

fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot(1, 1, 1)

AOD['volcanicDeltaFunc'] = ((AOD['AOD'].diff() > 0.01) &
                            (AOD['AOD'].diff().shift(1) < 0.01) &
                            (AOD['AOD'].diff().shift(2) < 0.01) &
                            (AOD['AOD'].diff().shift(3) < 0.01) &
                            (AOD['AOD'].diff().shift(4) < 0.01) &
                            (AOD['AOD'].diff().shift(5) < 0.01)
                           )

ax.plot(indexToYear(ta_diff.index),ta_diff.values)
ax.plot(AOD['time_yr'],AOD['AOD'],label='volcanic AOD',color='red')
ax.plot(AOD['time_yr'],-0.35+0.05*AOD['volcanicDeltaFunc'],label='volcanic AOD',color='orange')
ax.plot(indexToYear(ta_diff.index),-0.45+0.05*(ta_diff < -0.06),label='volcanic AOD',color='purple')

plt.xlabel('time [years]')
plt.ylabel('diff(temperature) OR volcanic activity')

plt.text(5,0.1,'Volcanic activity',color='red')
plt.text(5,-0.1,'global diff(Temperature)',color='tab:blue')
plt.text(5,-0.33,'True events',color='orange')
plt.text(5,-0.43,'Events passing threshold',color='purple')
plt.text(775,-0.43,'FP',color='purple')
ax.set_xlim(0,1000);

This simple differencing technique found all eight "major events", two "minor events" (out of 14), and had one false positive - not bad for a couple of simple transformations! Let's see if we can improve a bit by picking out a subset of the data, namely the temperate and tropical zones.

In the end, this one does not do much better, but it might give you a little bit more information about the location (southern or northern hemisphere) of the volcanic activity:

In [ ]:
s_hemi = HelperModules.getLatSlice(T2m_R1,-61,0)
s_hemi_avg = EngineeredFeatures.getGlobalTemperatureAverageSeries(s_hemi['T2m'])
ta_diff_s = s_hemi_avg.rolling(window=5).mean().diff().fillna(0).rolling(window=4).mean()

n_hemi = HelperModules.getLatSlice(T2m_R1,0,61)
n_hemi_avg = EngineeredFeatures.getGlobalTemperatureAverageSeries(n_hemi['T2m'])
ta_diff_n = n_hemi_avg.rolling(window=5).mean().diff().fillna(0).rolling(window=4).mean()

fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot(1, 1, 1)
ax.plot(indexToYear(ta_diff_s.index),ta_diff_s.values,color='tab:blue')
ax.plot(indexToYear(ta_diff_n.index),ta_diff_n.values,color='lightblue')

ax.plot(AOD['time_yr'],AOD['AOD'],label='volcanic AOD',color='red')
ax.plot(AOD['time_yr'],-0.35+0.05*AOD['volcanicDeltaFunc'],label='volcanic AOD',color='orange')
so_or_north = -0.45+0.05*(((ta_diff_s < -0.070) & (ta_diff_n < -0.020)) |
                          ((ta_diff_n < -0.070) & (ta_diff_s < -0.020)) |
                          (ta_diff_s + ta_diff_n < -0.10))
ax.plot(indexToYear(ta_diff_s.index),so_or_north,label='volcanic AOD',color='purple')

plt.text(5,0.1,'Volcanic activity',color='red')
plt.text(5,-0.1,'Southern hemisphere',color='tab:blue')
plt.text(5,-0.15,'Northern hemisphere',color='lightblue')
plt.text(5,-0.33,'True events',color='orange')
plt.text(5,-0.43,'Events passing threshold',color='purple')
plt.text(965,-0.43,'FP',color='purple')

plt.xlabel('time [years]')
plt.ylabel('diff(temperature) OR volcanic activity')
ax.set_xlim(0,1000);

Using Machine Learning to boost the signal
==========

We seem to have gotten as far as we could by just playing with the data, so now we have two options: we could spend a lot more time to try to understand the data, tinkering and finding more features, or we could throw things into Machine Learning and see if it can beat us. The latter will be faster, albeit with maybe fewer insights. But since part of the goal is to get more experience with Machine Learning, let's try this approach.

Setting up the training data
------

First, we split the data into windows, each with 18 years, and then label them as true volcano events or false ones. To slightly boost our statistics, we consider a year to host a true volcano event if it's within 1 year of the start of the volcanic activity.

Our background dataset consists of any data window, provided it has 18 years in the window and the beginning is at least 8 years away from the start of a volcanic event. This provides us with plenty of background events.

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot(1, 1, 1)

window_yr = 18

volcano_years = AOD[AOD['volcanicDeltaFunc'] == True]['time_yr'][:-1]
years_volcano_true = np.asarray(volcano_years.values.tolist() +
                                (volcano_years.values + 1).tolist() +
                                (volcano_years.values - 1).tolist()
                               )*10000+716.0

non_volcano = (AOD['volcanicDeltaFunc'] == False)
for i in range(8) :
    non_volcano = (non_volcano & (AOD['volcanicDeltaFunc'].shift(-i) == False))
for i in range(8) :
    non_volcano = (non_volcano & (AOD['volcanicDeltaFunc'].shift(i) == False))

non_volcano_years = AOD[non_volcano]['time_yr'][:-19]
years_volcano_false = non_volcano_years.values*10000+716.0

all_years = AOD['time_yr'][:-19]
years_all_t2m = all_years.values*10000+716.0

x_sig,x_sig_yr = [],[]
x_bkg,x_bkg_yr = [],[]
x_all,x_all_yr = [],[]

ax.plot(AOD['time_yr'],AOD['AOD'],label='volcanic AOD',color='red')

for i in years_volcano_true :
    ta_diff_subset = ta_diff[i:i+window_yr*10000]
    year = np.round(i/10000)
    ax.plot(indexToYear(ta_diff_subset.index),ta_diff_subset.values)
    x_sig.append(ta_diff_subset.values.tolist())
    x_sig_yr.append(year)

for i in non_volcano_years.values*10000+716.0 :
    year = np.round(i/10000)
    ta_diff_subset = ta_diff[i:i+window_yr*10000]
    x_bkg.append(ta_diff_subset.values.tolist())
    x_bkg_yr.append(year)

for i in years_all_t2m :
    year = np.round(i/10000)
    ta_diff_subset = ta_diff[i:i+window_yr*10000]
    x_all.append(ta_diff_subset.values.tolist())
    x_all_yr.append(year)
    
plt.xlabel('time [years]')
plt.ylabel('diff(temperature) OR volcanic activity')

plt.text(5,0.1,'Volcanic activity',color='red')
plt.text(5,0.15,'signal events (colored)',color='black')
ax.set_xlim(0,1000)

x_sig = np.array(list(a for a in x_sig))
print("Shape of signal events:",x_sig.shape)

x_bkg = np.array(list(a for a in x_bkg))
print("Shape of bkg events:",x_bkg.shape)

x_all = np.array(list(a for a in x_all))

In [ ]:
all_train = np.append(x_sig,x_bkg,axis=0)
all_train_labels = np.append([True]*x_sig.shape[0],[False]*x_bkg.shape[0])
all_train_years = np.append(x_sig_yr,x_bkg_yr)

df_train = pd.DataFrame(all_train_labels,columns=['label'])
df_train['year'] = all_train_years

df_test = pd.DataFrame(x_all_yr,columns=['year'])

Training the model
----------

As a model, let's use keras, via tensorflow, and pick a neural network with two hidden layers, each with 5 nodes. We finish with a sigmoid function layer for a binary output:

In [ ]:
import tensorflow as tf
print('Tensorflow version:',tf.__version__)

np.random.seed(1)
tf.random.set_seed(1)

model = tf.keras.models.Sequential([
#   tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(5, name='first_dense', input_shape=(19,), activation='relu'),
    tf.keras.layers.Dense(5, name='middle_dense', activation='relu'),
#   tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, name='last_dense', activation='sigmoid') # sigmoid for binary; softmax for multi?
    ])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(
    all_train,
    all_train_labels,
    batch_size=64,
    epochs=20,
    verbose=0);

Evaluating the Model
-------
Taking a look at the confusion matrix below, it seems like our model has done a terrible job. However, if we plot the discriminant itself, there seems to be plenty of discriminating power. We can set a discriminating value by hand to select the signal acceptance / false positive rate that we want.

In [ ]:
from sklearn.metrics import confusion_matrix
df_train['binary_prediction'] = model.predict_classes(all_train)
confusion_matrix(df_train['binary_prediction'],all_train_labels)

In [ ]:
df_train['discriminant'] = model.predict(all_train)
df_test['discriminant'] = model.predict(x_all)
rng=[0.15,0.47]

plt.hist(df_train['discriminant'],bins=40,label='all',density=False,histtype='stepfilled',lw=2,range=rng,color='lightblue')
plt.hist(df_train[df_train['label'] ==1]['discriminant'],bins=40,label='sig',density=False,histtype='step',lw=2,range=rng)
plt.hist(df_train[df_train['label'] ==0]['discriminant'],bins=40,label='bkg',density=False,histtype='step',lw=2,range=rng)

plt.legend()
plt.yscale('log')
plt.xlabel('discriminant value')
plt.ylabel('events');

Finally, let's plot our chosen signal events and compare them to our naive threshold method:

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot(1, 1, 1)

ax.plot(indexToYear(ta_diff.index),ta_diff.values)
ax.plot(AOD['time_yr'],AOD['AOD'],label='volcanic AOD',color='red')
ax.plot(AOD['time_yr'],-0.35+0.05*AOD['volcanicDeltaFunc'],label='volcanic AOD',color='orange')
ax.plot(indexToYear(ta_diff.index),-0.45+0.05*(ta_diff < -0.06),label='volcanic AOD',color='purple')
ax.plot(df_test['year'],-0.55+0.05*(df_test['discriminant'] > 0.334),color='green')

plt.text(5,0.1,'Volcanic activity',color='red')
plt.text(5,-0.1,'global diff(Temperature)',color='tab:blue')
plt.text(5,-0.33,'True events',color='orange')
plt.text(5,-0.43,'Events passing threshold',color='purple')
plt.text(5,-0.53,'Keras NN',color='green')
plt.text(775,-0.43,'FP',color='purple')
plt.text(772,-0.53,'FP',color='green')

plt.xlabel('time [years]')
plt.ylabel('diff(temperature) OR volcanic activity')
ax.set_xlim(0,1000);

Okay - the simple neural network did pretty well! It managed to get all 8 major events, plus 4 minor events, with only 1 false positive. You'll remember that the naive threshold-setting method ("Events passing threshold") caught only 2 minor events, also with one false positive. Even with 0 tweaking, the neural net seems to do quite well.

Wrapping Up, for now
-----------
We could continue to tweak this NN -- perhaps giving it more fine-grained information on temperatures in different regions, or using a more targeted ML approach such as **Long short-term memory networks (LSTM)**. However, to do this properly we would probably need a larger dataset, to avoid things like overtraining. Thus, I think the main takeaway of this exercise is this: that a naive approach gets you pretty far in terms of signal detection, and machine learning can take you a bit farther... however at the cost of obscuring the mechanism that bring such success.